<a href="https://colab.research.google.com/github/RuoxueWang-lab/Linear-NN-model-iris-dataset/blob/main/Linear_NN_model_for_Iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f

In [3]:
class Model(nn.Module):

  #Create our layers
  def __init__(self, in_features=4, l1_count=10, l2_count=10, out_features=20) -> None:
    super().__init__()
    self.l1 = nn.Linear(in_features, l1_count)
    self.l2 = nn.Linear(l1_count, l2_count)
    self.output = nn.Linear(l2_count, out_features)


  #Define our forward propagation
  def forward(self,x):
    x = f.relu(self.l1(x))
    x = f.relu(self.l2(x))
    x = self.output(x)

    return x


In [4]:
#Create our Model instance
model = Model()

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
url = "https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
iris_df = pd.read_csv(url)
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


##Ordinal encoding


In [7]:
from sklearn.preprocessing import OrdinalEncoder

In [8]:
encoder = OrdinalEncoder()
iris_df["species"] = encoder.fit_transform(iris_df[["species"]])
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [9]:
X = iris_df.drop("species", axis=1).values
y = iris_df["species"].values

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [25]:
#Set up the criterion to error
criterior = nn.CrossEntropyLoss()
#Set up out optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [36]:
#Do Backpropagation
epochs = 200
for i in range(0, epochs):
  y_pred = model.forward(X_train)
  error = criterior(y_pred, y_train)
  optimizer.zero_grad()
  error.backward()
  optimizer.step()